# Modelo Utilizando Arboles De Decisión

## Importación de las librerias

In [187]:
import numpy as np
import pandas as pd
import tensorflow as tf
import numpy as np

import seaborn as sns
sns.set_style('whitegrid')
import matplotlib.pyplot as plt

%matplotlib inline
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score

from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

from tensorflow import keras


from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
from keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasClassifier
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans




from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import numpy as np


## Importamos el dataset

In [188]:
dataset = pd.read_csv('/content/datasetParcial.csv')

## Preprocesamiento de los datos

In [189]:
X = dataset.iloc[:, 1:-1].values
y = dataset.iloc[:, -1].values

## Particion de los datos entre el conjunto de entrenamiento y el conjunto para test

In [190]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [191]:
print(X_train)

[[1.98900000e+01 3.54000000e+01 0.00000000e+00 5.29000000e+02
  5.08894379e-03]
 [2.00000000e+01 3.57000000e+01 0.00000000e+00 5.40000000e+02
  5.16782401e-03]
 [2.07900000e+01 2.62000000e+01 0.00000000e+00 1.49800000e+03
  3.97467591e-03]
 ...
 [2.05000000e+01 2.37000000e+01 0.00000000e+00 5.81500000e+02
  3.52920391e-03]
 [2.01000000e+01 3.22000000e+01 0.00000000e+00 5.69666667e+02
  4.68652701e-03]
 [2.15000000e+01 2.88233333e+01 0.00000000e+00 6.22333333e+02
  4.57184298e-03]]


In [192]:
print(X_test)

[[2.03900000e+01 3.27900000e+01 0.00000000e+00 6.59000000e+02
  4.86008397e-03]
 [2.10000000e+01 3.37000000e+01 0.00000000e+00 1.50000000e+03
  5.18898657e-03]
 [2.00000000e+01 3.05000000e+01 0.00000000e+00 7.16500000e+02
  4.40975111e-03]
 ...
 [2.06000000e+01 2.19700000e+01 6.00000000e+00 9.26750000e+02
  3.29057323e-03]
 [2.01000000e+01 3.35000000e+01 0.00000000e+00 5.51000000e+02
  4.87721823e-03]
 [2.06000000e+01 2.50000000e+01 0.00000000e+00 5.50000000e+02
  3.74712771e-03]]


In [193]:
print(y_train)

[0 0 0 ... 0 0 0]


In [194]:
print(y_test)

[0 0 0 ... 0 0 0]


## Escalando las caracteristicas

In [195]:
sc = StandardScaler()
X_train[:, 1:-1] = sc.fit_transform(X_train[:, 1:-1])
X_test[:, 1:-1] = sc.transform(X_test[:, 1:-1])

Inicializando la ANN

In [196]:
ann = tf.keras.models.Sequential()

Agregar la capa de entrada y la primera capa oculta

In [197]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

Agregando la segunda capa oculta

In [198]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))


Agregar la capa de salida

In [199]:
ann.add(tf.keras.layers.Dense(units=1))

Compilando la RNA

In [200]:
ann.compile(optimizer = 'adam', loss = 'mean_squared_error')

## Buscar hiperparámetros en la ANN

In [201]:
param_distribs = {
  "n_hidden": [0, 1, 2, 3],
  "n_neurons":np.arange(1,10) ,
  "learning_rate": reciprocal(3e-4,3e-2)
}


In [202]:
ann_model = KerasClassifier(build_model, n_hidden=param_distribs["n_hidden"],
                            n_neurons=param_distribs["n_neurons"], learning_rate=param_distribs["learning_rate"],
                            verbose = 1)

In [203]:
rnd_search_cv = RandomizedSearchCV(ann_model, param_distribs, n_iter=3, cv=3)

In [204]:
rnd_search_cv.fit(X_train, y_train, epochs=100, validation_split = 0.05, callbacks=[EarlyStopping(patience=10)]) # cambiar Epochs por un numero mayor para mejorar obtimizado

Epoch 1/100
155/155 [==============================] - 1s 4ms/step - loss: 0.5791 - accuracy: 0.7913 - val_loss: 0.5154 - val_accuracy: 0.8038
Epoch 2/100
155/155 [==============================] - 0s 3ms/step - loss: 0.5139 - accuracy: 0.7913 - val_loss: 0.4796 - val_accuracy: 0.8038
Epoch 3/100
155/155 [==============================] - 0s 2ms/step - loss: 0.4662 - accuracy: 0.7913 - val_loss: 0.4162 - val_accuracy: 0.8038
Epoch 4/100
155/155 [==============================] - 0s 3ms/step - loss: 0.3725 - accuracy: 0.7988 - val_loss: 0.3092 - val_accuracy: 0.8500
Epoch 5/100
155/155 [==============================] - 0s 3ms/step - loss: 0.2651 - accuracy: 0.8850 - val_loss: 0.2204 - val_accuracy: 0.9308
Epoch 6/100
155/155 [==============================] - 0s 3ms/step - loss: 0.1915 - accuracy: 0.9528 - val_loss: 0.1549 - val_accuracy: 0.9577
Epoch 7/100
155/155 [==============================] - 0s 3ms/step - loss: 0.1423 - accuracy: 0.9789 - val_loss: 0.1151 - val_accuracy: 0.9923

RandomizedSearchCV(cv=3,
                   estimator=KerasClassifier(learning_rate=<scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x78a88251e6b0>, model=<function build_model at 0x78a8824bf130>, n_hidden=[0, 1, 2, 3], n_neurons=array([1, 2, 3, 4, 5, 6, 7, 8, 9])),
                   n_iter=3,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x78a88251e6b0>,
                                        'n_hidden': [0, 1, 2, 3],
                                        'n_neurons': array([1, 2, 3, 4, 5, 6, 7, 8, 9])})

In [205]:
rnd_search_cv.best_params_

{'learning_rate': 0.0007227049562603265, 'n_hidden': 3, 'n_neurons': 3}

In [206]:
rnd_search_cv.best_score_

0.9935904336596378

In [207]:
ann = rnd_search_cv.best_estimator_
ann

KerasClassifier(
	model=<function build_model at 0x78a8824bf130>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
	n_hidden=3
	n_neurons=3
	learning_rate=0.0007227049562603265
	class_weight=None
)

 ### keras optimización

Función para construir y compilar un modelo Keras.

In [208]:
def build_model(n_hidden=1, n_neurons=6, learning_rate=3e-3, input_shape=X_train.shape[1]):
  model = Sequential()
  model.add(InputLayer(input_shape=input_shape))
  for layer in range(n_hidden):
    model.add(Dense(n_neurons, activation="relu"))
  model.add(Dense(1, activation='sigmoid'))
  optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
  model.compile(loss='binary_crossentropy', optimizer=optimizer,metrics = ['accuracy'])
  return model

In [209]:
!pip install scikeras

## Entrenamiento del modelo ANN en el conjunto de entrenamiento

In [210]:
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=25)
ann.fit(x=X_train, y=y_train, validation_data=(X_test, y_test),
       batch_size=32, epochs=400, callbacks=[early_stop], verbose=0)

Epoch 137: early stopping


### Prediciendo los resultados del test

In [211]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

61/61 [==============================] - 0s 1ms/step
[[0 0]
 [0 0]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Evaluación de las metricas

##### Realizando la matriz de confusión

In [212]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1515   11]
 [   4  421]]


0.9923116350589442

##### Accuracy Score

In [213]:
accuracy_score(y_test, y_pred)

0.9923116350589442

##### Precision

In [214]:
# Valores desde la matriz de confusión
# TP: Verdaderos Positivos
# TN: Verdaderos Negativos
# FP: Falsos Positivos
# FN: Falsos Negativos

TP = 419  # Verdaderos Positivos
TN = 1515  # Verdaderos Negativos
FP = 11  # Falsos Positivos
FN = 6  # Falsos Negativos

# Cálculo de la precisión
precision = TP / (TP + FP)
print("Precisión:", precision)

Precisión: 0.9744186046511628


##### Recall

In [215]:
recall = TP / (TP + FN)
print("Recall:", recall)

Recall: 0.9858823529411764


##### F1-Score

In [216]:
f1_score = 2 * (precision * recall) / (precision + recall)
print("Puntaje F1:", f1_score)

Puntaje F1: 0.9801169590643274


####Score de redes neuronales

In [236]:
1-ann.evaluate(X_test, y_test)

61/61 [==============================] - 0s 3ms/step - loss: 0.0075


0.9925450761802495

#### Calculamos la tasa de error

In [217]:
1 - accuracy_score(y_test, y_pred)

0.0076883649410558474

## Obtimizado sin reduccion

In [218]:
ann.fit(X_train, y_train, batch_size=32, epochs=10)#obtimisado

Epoch 1/10
244/244 [==============================] - 1s 2ms/step - loss: 0.0055
Epoch 2/10
244/244 [==============================] - 1s 2ms/step - loss: 0.0056
Epoch 3/10
244/244 [==============================] - 0s 2ms/step - loss: 0.0055
Epoch 4/10
244/244 [==============================] - 0s 2ms/step - loss: 0.0055
Epoch 5/10
244/244 [==============================] - 0s 2ms/step - loss: 0.0056
Epoch 6/10
244/244 [==============================] - 1s 3ms/step - loss: 0.0055
Epoch 7/10
244/244 [==============================] - 1s 3ms/step - loss: 0.0057
Epoch 8/10
244/244 [==============================] - 1s 3ms/step - loss: 0.0054
Epoch 9/10
244/244 [==============================] - 1s 3ms/step - loss: 0.0053
Epoch 10/10
244/244 [==============================] - 1s 2ms/step - loss: 0.0054


Prediciendo los resultados del test

In [219]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

61/61 [==============================] - 0s 2ms/step
[[0 0]
 [0 0]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Evaluación de las metricas

##### Realizando la matriz de confusión

In [220]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)


[[1515   11]
 [   4  421]]


##### Accuracy Score

In [221]:
accuracy_score(y_test, y_pred)

0.9923116350589442

##### Precision

In [222]:
# Valores desde la matriz de confusión
# TP: Verdaderos Positivos
# TN: Verdaderos Negativos
# FP: Falsos Positivos
# FN: Falsos Negativos

TP = 419  # Verdaderos Positivos
TN = 1515  # Verdaderos Negativos
FP = 11  # Falsos Positivos
FN = 6  # Falsos Negativos

# Cálculo de la precisión
precision = TP / (TP + FP)
print("Precisión:", precision)

Precisión: 0.9744186046511628


##### Recall

In [223]:
recall = TP / (TP + FN)
print("Recall:", recall)

Recall: 0.9858823529411764


##### F1-Score

In [224]:
f1_score = 2 * (precision * recall) / (precision + recall)
print("Puntaje F1:", f1_score)

Puntaje F1: 0.9801169590643274


####Score de redes neuronales

In [237]:
1-ann.evaluate(X_test, y_test)

61/61 [==============================] - 0s 1ms/step - loss: 0.0075


0.9925450761802495

#### Calculamos la tasa de error

In [225]:
1 - accuracy_score(y_test, y_pred)

0.0076883649410558474

##Nos creamos un pipeline para combinar la ejecución, primero K-Means y luego Redes neuronales


In [246]:
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=25)
ann.fit(x=X_train, y=y_train, validation_data=(X_test, y_test),
       batch_size=32, epochs=400, callbacks=[early_stop], verbose=0)

Epoch 27: early stopping


In [251]:
from sklearn.decomposition import PCA
# Asegura que input_dim coincida con el número de dimensiones después de PCA
input_dim = 2

# Define un modelo de red neuronal
def build_neural_network_model(input_dim):
    model = Sequential()
    model.add(Dense(64, input_dim=input_dim, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
    return model

# Define el número de componentes principales deseado
n_components = 2

# Crea instancias de PCA
pca = PCA(n_components=n_components)

# Crea instancias de KerasClassifier con tu modelo
neural_network_classifier = KerasClassifier(build_fn=build_neural_network_model, input_dim=n_components, epochs=10, batch_size=32)

# Define el pipeline que incluye PCA, KMeans y el clasificador de redes neuronales
pipeline = Pipeline([
    ("kmeans", KMeans(n_clusters=60, random_state=42)),
    ("neural_network", neural_network_classifier)
])



In [252]:
# Ajusta el pipeline con tus datos de entrenamiento
pipeline.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


ValueError: ignored